In [3]:
# EXTRACT 
import awswrangler as awr
import logging
import pandas as pd
import os
import numpy as np
import datetime as dt
import openpyxl

logging.basicConfig(
    level=logging.INFO,  # Exibe mensagens a partir de INFO
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler()  # Garante logs no console
    ]
)

logging.info('\n ----------------------------------------------------------------------------------')
logging.info('\n Executando Rotina: Movimentação de Placas')

class ETL_boards:

#EXTRACT
    def __init__(self):
        self.path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes"
        self.today = pd.Timestamp.today().date()
        self.df_ativacoes = None
        self.df_cancelamentos = None
        self.df_conferencia = None
        self.df_final_ativacoes = None
        self.df_final_cancelamentos = None
        if self.today.weekday() == 0:  # 
            self.yesterday = self.today - pd.Timedelta(days=3)
        else:
            self.yesterday = self.today - pd.Timedelta(days=1)
       

    def extract_all_ativacoes(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_ATIVOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            self.df_ativacoes = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de ativações extraída com sucesso!')
            return self.df_ativacoes

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de ativações: {e}')
            return None

    def extract_all_cancelamentos(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_CANCELADOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            self.df_cancelamentos = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de cancelamentos extraída com sucesso!')
            return self.df_cancelamentos

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de cancelamentos: {e}')
            return None
    
    def extract_conf_boards(self):

        try:

            dir_query = os.path.join(self.path,'sql', 'listagem_mestra.sql')

            with open(dir_query, 'r') as file:
                query = file.read()

            self.df_conferencia = awr.athena.read_sql_query(query, database='silver')
        
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Relatorio conferência  - Dados Extraidos com sucesso!')

            return self.df_conferencia

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao Extrair relatorio conferência: {e}')



2025-09-17 15:44:14,932 - INFO - 
 ----------------------------------------------------------------------------------
2025-09-17 15:44:14,932 - INFO - 
 Executando Rotina: Movimentação de Placas


In [15]:

df_conferencia = etl.extract_conf_boards()

2025-09-17 16:21:04,795 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:05,720 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:07,076 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:08,379 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:09,352 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:11,336 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:13,137 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:14,996 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:16,886 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:18,714 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:21:20,

In [18]:
etl = ETL_boards()
df_ativ_all_boards = etl.extract_all_ativacoes()

2025-09-17 16:28:51,097 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:28:51,970 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:28:53,321 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:28:54,715 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:28:55,656 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:28:57,487 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:28:59,340 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:29:01,196 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:29:03,057 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:29:04,952 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-09-17 16:29:06,

In [9]:
today = pd.Timestamp.today().date()
yesterday = today - pd.Timedelta(days=1)

df_ativ_all_boards_unique = df_ativ_all_boards.drop_duplicates(subset='conjunto')
#df_ativ_all_boards_unique['data_ativacao'] = pd.to_datetime(df_ativ_all_boards_unique['data_ativacao']) 
df_ativ_all_boards_unique_yesterday = df_ativ_all_boards_unique[df_ativ_all_boards_unique['data_ativacao']==yesterday]
df_ativ_all_boards_unique_yesterday.shape[0]

43

In [19]:


# TRANSFORM
def board_status_treatment( df, df_conf, status_filter_list):

    try:
        if not df.empty:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(df.shape)
            row_count = 0
            for idx, row in df.iterrows():
                row_count += 1
                df_verification = df_conf[
                    (df_conf['chassi'] == row['chassi']) & (df_conf['beneficio'] == row['beneficio'])
                ].sort_values(by='data_ativacao', ascending=True)

                if not df_verification.empty and len(df_verification['empresa'].values) > 1:
                    hist_datas_ativacao = sorted(df_verification['data_ativacao_beneficio'].dropna().drop_duplicates().unique())

                    if len(hist_datas_ativacao) > 1:
                        penultimo_registro_data = hist_datas_ativacao[-2]
                        verification_penultima_row = df_verification.loc[df_verification['data_ativacao_beneficio'] == penultimo_registro_data]
                        
                        if verification_penultima_row['status_beneficio'].values[0] not in status_filter_list:
                            if verification_penultima_row['empresa'].values[0] != row['empresa']:
                                df.at[idx, 'status_beneficio'] = 'MIGRAÇÃO'
                                df.at[idx, 'migration_from'] = verification_penultima_row['empresa'].values[0]
                            else:
                                df.at[idx, 'status_beneficio'] = 'RENOVAÇÃO'
                                df.at[idx, 'migration_from'] = 'NULL'
                        else:
                            # today = dt.datetime.today()
                            # hist_datas_atualizacao = sorted(df_verification['data_atualizacao'].dropna().drop_duplicates().unique())
                            # penultimo_registro_data_atualizacao = hist_datas_atualizacao[-2]
                            # if today - penultimo_registro_data_atualizacao > dt.timedelta(days=30):
                                df.at[idx, 'status_beneficio'] = 'REATIVAÇÃO'
                                df.at[idx, 'migration_from'] = 'NULL'
                                
                            # else:
                            #     if verification_penultima_row ['empresa'].values[0] != row['empresa']:
                            #         df.at[idx, 'status_beneficio'] = 'MIGRAÇÃO'
                            #         df.at[idx, 'migration_from'] = verification_penultima_row['empresa'].values[0]
                            #     else:
                            #         df.at[idx, 'status_beneficio'] = 'RENOVAÇÃO'
                            #         df.at[idx, 'migration_from'] = 'NULL'
                                
                    else:
                        df.at[idx, 'status_beneficio'] = 'ATIVO'
                        df.at[idx, 'migration_from'] = 'NULL'
                else:
                    df.at[idx, 'status_beneficio'] = 'ATIVO'
                    df.at[idx, 'migration_from'] = 'NULL'

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Total de linhas processadas: {row_count}')

        else:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('Nnehum registro de ativações para tratamento de dados. Dataframe vazio!')


    except Exception as e:

        logging.info('\n ----------------------------------------------------------------------------------')
        logging.info(f'Falha no tratamento de status das placas ativadas. Revise o código: {e}')

    return df

# DEFININDO DATAFRAMES VAZIOS 
df_final_ativacoes = pd.DataFrame()


# TRANSFORMANDO DF_ATIV E SEGMENTANDO POR EMPRESA


df_ativ_all_boards['data_ativacao_beneficio'] = pd.to_datetime(df_ativ_all_boards['data_ativacao_beneficio']).dt.date

df_ativ_all_boards['beneficio'] = (
    df_ativ_all_boards['beneficio']
    .replace('REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO', 'CASCO (VEÍCULO)')
    .replace('REPARAÇÃO OU REPOSIÇÃO DO (SEMI)REBOQUE', 'CASCO (R/SR)')
    .replace('REPARAÇÃO OU REPOSIÇÃO DO COMPLEMENTO', 'CASCO (COMPLEMENTO)')
)



In [20]:
today = pd.Timestamp.today().date()
yesterday = today - pd.Timedelta(days=1)

df_ativ_all_boards_unique = df_ativ_all_boards.drop_duplicates(subset='conjunto')
#df_ativ_all_boards_unique['data_ativacao'] = pd.to_datetime(df_ativ_all_boards_unique['data_ativacao']) 
df_ativ_all_boards_unique_yesterday = df_ativ_all_boards_unique[df_ativ_all_boards_unique['data_ativacao']==yesterday]
df_ativ_all_boards_unique_yesterday.shape[0]

43

In [ ]:

today = pd.Timestamp.today().date()
yesterday = today - pd.Timedelta(days=1)

# CONCATENANDO E CRIANDO COLUNA DE MIGRAÇÃO (MIGRATION_FROM) 
df_ativ_all_boards2 = df_ativ_all_boards

if not df_ativ_all_boards2.empty:
    df_ativ_all_boards2['migration_from'] = np.nan

# CRIANDO LISTA DE VERIFICAÇÃOST DE PLACAS MIGRADAS (STATUS)
status_filter_list = ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO']

# PEGANDO DADOS DE ATIVAÇÃO DO DIA ANTERIOR E TRATANDO
if not df_ativ_all_boards2.empty:

    df_ativos_dia_anterior = df_ativ_all_boards2[df_ativ_all_boards2['data_ativacao'] == yesterday]
    df_ativacoes_dia_anterior_ranking_tratado = board_status_treatment(
        df_ativos_dia_anterior, df_conferencia, status_filter_list
    )
    df_ativ_all_boards2 = pd.concat([df_ativacoes_dia_anterior_ranking_tratado, df_ativ_all_boards2])
    df_ativ_all_boards2 = df_ativ_all_boards2.drop_duplicates(subset='chassi', keep='first')

# DEFININDO COLUNAS QUE SERÃO UTILIZADAS NO DATAFRAME FINAL
df_ativ_all_boards2 = df_ativ_all_boards2[[
    'placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria', 'matricula', 'conjunto', 'unidade', 'consultor', 'status_beneficio', 
    'cliente', 'data_registro', 'data_ativacao', 'data_ativacao_beneficio', 'suporte', 'data_filtro', 'empresa', 'migration_from'
]]


2025-09-17 16:43:37,958 - INFO - 
 ----------------------------------------------------------------------------------
2025-09-17 16:43:37,960 - INFO - (354, 26)


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_19952\1146755659.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NULL' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'migration_from'] = 'NULL'
2025-09-17 16:44:10,134 - INFO - 
 ----------------------------------------------------------------------------------
2025-09-17 16:44:10,136 - INFO - Total de linhas processadas: 354


In [26]:
today = pd.Timestamp.today().date()
yesterday = today - pd.Timedelta(days=1)

df_ativ_all_boards_unique = df_ativ_all_boards2.drop_duplicates(subset='conjunto')
#df_ativ_all_boards_unique['data_ativacao'] = pd.to_datetime(df_ativ_all_boards_unique['data_ativacao']) 
df_ativ_all_boards_unique_yesterday = df_ativ_all_boards_unique[df_ativ_all_boards_unique['data_ativacao']==yesterday]
df_ativ_all_boards_unique_yesterday.shape[0]

43

In [ ]:

# TRATANDO DADOS NULOS NOS DATAFRAMES
df_ativ_all_boards['placa'] = df_ativ_all_boards['placa'].fillna('SEM-PLACA')
df_ativ_all_boards['chassi'] = df_ativ_all_boards['chassi'].fillna('NULL')
df_ativ_all_boards['id_placa'] = df_ativ_all_boards['id_placa'].fillna(0)
df_ativ_all_boards['id_veiculo'] = df_ativ_all_boards['id_veiculo'].fillna(0)
df_ativ_all_boards['id_carroceria'] = df_ativ_all_boards['id_carroceria'].fillna(0)
df_ativ_all_boards['matricula'] = df_ativ_all_boards['matricula'].fillna(0)
df_ativ_all_boards['conjunto'] = df_ativ_all_boards['conjunto'].fillna(0)
df_ativ_all_boards['unidade'] = df_ativ_all_boards['unidade'].fillna('NULL')
df_ativ_all_boards['consultor'] = df_ativ_all_boards['consultor'].fillna('NULL')
df_ativ_all_boards['status_beneficio'] = df_ativ_all_boards['status_beneficio'].fillna('NULL')
df_ativ_all_boards['cliente'] = df_ativ_all_boards['cliente'].fillna('NULL')
df_ativ_all_boards['data_registro'] = df_ativ_all_boards['data_registro'].fillna(pd.Timestamp('1900-01-01'))
df_ativ_all_boards['data_ativacao_beneficio'] = df_ativ_all_boards['data_ativacao_beneficio'].fillna(pd.Timestamp('1900-01-01'))
df_ativ_all_boards['suporte'] = df_ativ_all_boards['suporte'].fillna('NULL')
df_ativ_all_boards['data_filtro'] = df_ativ_all_boards['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
df_ativ_all_boards['empresa'] = df_ativ_all_boards['empresa'].fillna('NULL')
df_ativ_all_boards['migration_from'] = df_ativ_all_boards['migration_from'].fillna('NULL')

df_final_cancelamentos['placa'] = df_final_cancelamentos['placa'].fillna('SEM-PLACA')
df_final_cancelamentos['chassi'] = df_final_cancelamentos['chassi'].fillna('NULL')
df_final_cancelamentos['id_placa'] = df_final_cancelamentos['id_placa'].fillna(0)
df_final_cancelamentos['id_veiculo'] = df_final_cancelamentos['id_veiculo'].fillna(0)
df_final_cancelamentos['id_carroceria'] = df_final_cancelamentos['id_carroceria'].fillna(0)
df_final_cancelamentos['matricula'] = df_final_cancelamentos['matricula'].fillna(0)
df_final_cancelamentos['conjunto'] = df_final_cancelamentos['conjunto'].fillna(0)
df_final_cancelamentos['unidade'] = df_final_cancelamentos['unidade'].fillna('NULL')
df_final_cancelamentos['status'] = df_final_cancelamentos['status'].fillna('NULL')
df_final_cancelamentos['cliente'] = df_final_cancelamentos['cliente'].fillna('NULL')
df_final_cancelamentos['data'] = df_final_cancelamentos['data'].fillna(pd.Timestamp('1900-01-01'))
df_final_cancelamentos['data_cancelamento'] = df_final_cancelamentos['data_cancelamento'].fillna(pd.Timestamp('1900-01-01'))
df_final_cancelamentos['usuario_cancelamento'] = df_final_cancelamentos['usuario_cancelamento'].fillna('NULL')
df_final_cancelamentos['data_filtro'] = df_final_cancelamentos['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
df_final_cancelamentos['empresa'] = df_final_cancelamentos['empresa'].fillna('NULL')
df_final_cancelamentos['migracao'] = df_final_cancelamentos['migracao'].fillna('NULL')
df_final_cancelamentos['renegociacao'] = df_final_cancelamentos['renegociacao'].fillna('NULL')
df_final_cancelamentos['data_substituicao'] = df_final_cancelamentos['data_substituicao'].fillna(pd.Timestamp('1900-01-01'))

# salvar no estado do objeto para uso no LOAD
self.df_ativ_all_boards = df_ativ_all_boards
self.df_final_cancelamentos = df_final_cancelamentos




In [ ]:
# LOAD
def loading_files(self):

    try:
        # garantir dados transformados no estado
        if self.df_final_ativacoes is None or self.df_final_cancelamentos is None:
            if self.df_ativacoes is None:
                self.extract_all_ativacoes()
            if self.df_cancelamentos is None:
                self.extract_all_cancelamentos()
            if self.df_conferencia is None:
                self.extract_conf_boards()
            self.transforming_files()

        file_path = rf"C:\Users\raphael.almeida\Documents\Processos\placas_acompanhamento\template\placas_movimentacoes_{self.today}.xlsx"

        destination_dir = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas"
        destination_path = os.path.join(destination_dir, os.path.basename(file_path))

        destination_dir2 = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\bkp_activation"
        destination_path2 = os.path.join(destination_dir2, os.path.basename(file_path))

        os.makedirs(destination_dir, exist_ok=True)
        os.makedirs(destination_dir2, exist_ok=True)

        with pd.ExcelWriter(destination_path, engine='openpyxl') as writer:
            self.df_final_ativacoes.to_excel(writer, index=False, sheet_name='ATIVAÇÕES')
            self.df_final_cancelamentos.to_excel(writer, index=False, sheet_name='CANCELAMENTOS')

        with pd.ExcelWriter(destination_path2, engine='openpyxl') as writer:
            self.df_final_ativacoes.to_excel(writer, index=False, sheet_name='ATIVAÇÕES')
            self.df_final_cancelamentos.to_excel(writer, index=False, sheet_name='CANCELAMENTOS')

        logging.info('\n ----------------------------------------------------------------------------------')
        logging.info('\n Processo de Carregamento de Dados concluido com sucesso!')
    except Exception as e:
        logging.info('\n ----------------------------------------------------------------------------------')
        logging.info(f'\n Falha no processo de carregamento: {e}')

if __name__ == '__main__':
etl = ETL_boards()
etl.loading_files()



